In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation


print(tf.__version__)


2.10.0


In [4]:
# Pandas dataframe to open the the CSV file and get the Descriptions data
# Load the data from the file
df = pd.read_csv('network_data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'network_data.csv'

In [ ]:
#Tokenization -> Text Vectorization (preprocessing layer that maps text feuatures to integer sequences)

# Make the output integer data type
#Max_tokens parameter is a parameter that sets the maximum number of tokens/unique words in the vocabulary after the model gets preprocessed
# Setting it to vocab size means you lose no vocabuary
text_processor = tf.keras.layers.TextVectorization(max_tokens=vocab_size, output_mode=int)
text_processor.fit_on_texts(descriptions)

#Transform applies the vectorization to the input data right before it gets passed through
text_sequences = text_processor.transform(descriptions)

In [ ]:
#Padding -> Adds 0's to the shorter sequences so all the input sequences stay the same
# Framworks like tensorflow expect all inputs to be the same
# Post padding is always nessecary and important because when it applies before 
# the data, the model will take a while to learn and make it inefficient
padded_sequences = pad_sequences(text_sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
#Split the Data into an 80:20 split
train_data = desc_padded[:int(0.8 * len(desc_padded))]
train_labels = labels[:int(0.8 * len(desc_padded))]
test_data = desc_padded[int(0.8 * len(desc_padded)):]
test_labels = labels[int(0.8 * len(desc_padded)):]

### RNN Architecture
- RNN - Reccurent Neural Network - makes predictions based on multiple inputs based on the problem 
- Sequential in terms of a network means -> input to output
- RNNs have feedback loops which set the network apart
    - Feedback oops -> (weightOfLayer * CurrentValue) + (y1*weigthOfFeedback)

In [1]:
#Build the Model
model = Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
#Gated reccurent unit layer -> reccurent layer - return sequences is the feedback loop
model.add(tf.keras.layers.GRU(units=hidden_units, return_sequences=True))
model.add(tf.keras.layers.GRU(units=hidden_units))
model.add(LSTM(units=hidden_units))
model.add(tf.keras.layers.Dense(1, activation='Sigmoid'))

model.summary()

NameError: name 'Sequential' is not defined

In [ ]:
#Compile and train
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])